In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')


In [1]:
# %load_ext autoreload
# %autoreload 2

# !pip -q install sklearn
# !pip -q install turicreate

import pandas as pd
import numpy as np
import pickle
import gc

import turicreate as tc
from sklearn.model_selection import train_test_split

projectPath = "./co_filter"
gc.enable()

## Get Raw Data and parse useful data only

In [2]:
initial_co_filter_df_aquire = False
if initial_co_filter_df_aquire: # Parse datafram to customerID, ItemID, Rating where rating is just count of purchase
    # article = pd.read_csv("/content/gdrive/MyDrive/Capstone/cleaned_data/articles_clean.csv")
    # customer = pd.read_csv("/content/gdrive/MyDrive/Capstone/cleaned_data/customers_clean.csv")
    trans = pd.read_csv("./transactions_train.csv")

    customArticlePair = trans[["customer_id", "article_id"]] #.head(1000)
    countPlaceholder = pd.Series([1 for x in range(len(customArticlePair))])
    customArticlePair.loc[:, "rating"] = countPlaceholder # Add a column of purchase count
    # customArticlePair.to_csv("/content/gdrive/MyDrive/Capstone/co_filter/customer_article_pair.csv",index=False)
    countGroup = customArticlePair.groupby(["customer_id", "article_id"]).count() # count purchase numbers
    countGroup.to_csv(f"{projectPath}/customer_article_count.csv") # Save Counts to csv file

## Make ids continious

In [4]:
def index_continuous_encode(df, colName):
    ''' Encode original ids into continious ids for future pivoting '''
    current = 0
    idEncodeDict = {}
    for i in df.index:
        idOrigin = df.at[i, colName]
        if idOrigin not in idEncodeDict.keys():
            idEncodeDict[idOrigin] = current
            current += 1
        df.loc[i, colName] = idEncodeDict[idOrigin]
    return idEncodeDict

def index_encode(df, colName, savePath):
    idList = df[colName].unique()
    idMap = list(range(len(idList)))
    decodebook = dict(zip(idMap, idList))
    pickle_dump(f"{savePath}/{colName}_decoding_{nameClue}_user.pkl", decodebook)
    gc.collect()
    encodebook = dict(zip(idList, idMap))
    pickle_dump(f"{savePath}/customer_id_encoding_{nameClue}_user.pkl", encodebook)
    df.replace({colName: encodebook}, inplace=True)
    gc.collect()



def get_n_user_rating(df, n=-1):
    ''' Sample all movies viewed by n users, better partial prediction than random pick'''
    if n==-1: dfSampledUser = df['customer_id']
    else: dfSampledUser = df['customer_id'].sample(n=n)
    selected = df.loc[df['customer_id'].isin(dfSampledUser)].copy()
    return selected

def pickle_dump(path, d):
    with open(path, 'wb+') as f:
        pickle.dump(d, f)

In [ ]:
# TODO: Parse file into small portions and create dictionaries into portions

In [5]:
encode_all_ids = True
make_continuous = True
if encode_all_ids:
    userCount = -1 # all users
    nameClue = "all" if userCount == -1 else str(userCount)
    countDf = pd.read_csv(f"{projectPath}/customer_article_count.csv")
    dfSample = get_n_user_rating(countDf, userCount) # get all purchase of random n users
    # tester = dfSample.sort_values(by='rating', ascending=False).head(100)
    if make_continuous:
        index_encode(dfSample, 'customer_id', projectPath)
        index_encode(dfSample, 'article_id', projectPath)
    dfSample.to_csv(f"{projectPath}/sampled_{nameClue}_user_all_purchase.csv") # Save Counts to csv file

Error: Canceled future for execute_request message before replies were done

## Normalize purchase as recommendation rating

In [5]:
def pickle_load(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

if False: # Reminder, For future use
    encodes = {}
    encodes["customer_id"] = pickle_load(f"{projectPath}/customer_id_encoding_{getUserCount}_user.pkl")
    encodes["article_id"] = pickle_load(f"{projectPath}/article_id_encoding_{getUserCount}_user.pkl")

In [6]:
def normalize_purchase_count(df):
    df_matrix = pd.pivot_table(df, values='rating', index='customer_id', columns='article_id')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    print(d.head())
    # d.index.names = ['scaled_purchase_freq']
    # return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [3]:
normalize_rating = True
if normalize_rating:
    userCount = -1
    nameClue = "all" if userCount == -1 else str(userCount)
    dfSample = pd.read_csv(f"{projectPath}/sampled_{nameClue}_user_all_purchase.csv") # Save Counts to csv file
    
    df_matrix = pd.pivot_table(dfSample, values='rating', index='customer_id', columns='article_id')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    df_matrix_norm = df_matrix_norm.reset_index() # get customer_id out from being row index for unpivoting
    
    ratingNormDf = pd.melt(df_matrix_norm, id_vars=['customer_id'], value_name='rating_norm').dropna() # unpivot article_ids into tuples
    ratingNormDf.to_csv(f"{projectPath}/normalized_rating_triples.csv") # Save to csv file
    # display(ratingNormDf.head())

Error: Canceled future for execute_request message before replies were done

## Actual Recommendation

In [ ]:
def data_split(data, test_ratio=0.2):
    train, test = train_test_split(data, test_size =test_ratio)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [ ]:
split_data = True
if split_data:
    ratingNormDf = pd.read_csv(f"{projectPath}/normalized_rating_triples.csv")
    train_data_norm, test_data_norm = data_split(ratingNormDf)

In [ ]:
def createModel(train_data, simType, user_id, item_id, target):
    if simType == 'popularity':
        model = tc.popularity_recommender.create(train_data, user_id=user_id, item_id=item_id, target=target)
    elif simType == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, user_id=user_id, item_id=item_id, target=target, similarity_type='cosine')
    elif simType == 'pearson':
            model = tc.item_similarity_recommender.create(train_data, user_id=user_id, item_id=item_id, target=target, similarity_type='pearson')
    return model

user_id = 'customer_id'
item_id = 'article_id'
target = 'rating_norm'
simType = 'popularity'
usersToRecommend = list(set(ratingNormDf['customer_id'].tolist()))

model = createModel(train_data_norm, simType, user_id, item_id, target)
res = model.recommend(users=usersToRecommend, k=3)
res.print_rows(20)